In [1]:
%matplotlib inline

import os
import pickle
import numpy as np
import pandas as pd

	- instant: record index
	- dteday : date
	- season : season (1:springer, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2011, 1:2012)
	- mnth : month ( 1 to 12)
	- hr : hour (0 to 23)
	- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
	- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
	- hum: Normalized humidity. The values are divided to 100 (max)
	- windspeed: Normalized wind speed. The values are divided to 67 (max)
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

In [2]:
df = pd.read_csv('~/Desktop/py/data/day.csv')

df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
instant,731.0,366.000000,211.165812,1.000000,183.500000,366.000000,548.500000,731.000000
season,731.0,2.496580,1.110807,1.000000,2.000000,3.000000,3.000000,4.000000
yr,731.0,0.500684,0.500342,0.000000,0.000000,1.000000,1.000000,1.000000
mnth,731.0,6.519836,3.451913,1.000000,4.000000,7.000000,10.000000,12.000000
holiday,731.0,0.028728,0.167155,0.000000,0.000000,0.000000,0.000000,1.000000
weekday,731.0,2.997264,2.004787,0.000000,1.000000,3.000000,5.000000,6.000000
workingday,731.0,0.683995,0.465233,0.000000,0.000000,1.000000,1.000000,1.000000
weathersit,731.0,1.395349,0.544894,1.000000,1.000000,1.000000,2.000000,3.000000
temp,731.0,0.495385,0.183051,0.059130,0.337083,0.498333,0.655417,0.861667
atemp,731.0,0.474354,0.162961,0.079070,0.337842,0.486733,0.608602,0.840896


Dropping based on a priori knowledge. Instant is an index and increments like a date, so we can use that as a time variable.

In [3]:
from sklearn.model_selection import train_test_split as tts

features = [
    'instant', 'season', 'yr', 'mnth', 'holiday', 'weekday',
    'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed'
]

target = 'cnt'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [4]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [5]:
# OLS
from sklearn.linear_model import LinearRegression

# Instantiate the model
model = LinearRegression()
# Fit the model on the training data
model.fit(X_train, y_train)

# Using the fitted model, predict y-hats using the test set
yhat = model.predict(X_test)

# Calculate R^2 and MSE by comparing actual and fitted y
r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f}, MSE={:0.3f}".format(r2, me))

r2=0.807, MSE=718316.316


In [6]:
# Ben sets alphas for L2 and L1 regularization - default is (0.1, 1.0, 10.0) (start, stop, num)
# Larger values specify stronger regularization and Ben's options are much smaller than the default
# The model becomes more sparse as alpha increases
alphas = np.logspace(-10, 0, 200)

In [7]:
from sklearn.linear_model import RidgeCV

model = RidgeCV()
model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f}, MSE={:0.3f}, alpha={:0.3f}".format(r2, me, model.alpha_))

r2=0.807, MSE=720145.101, alpha=0.100


In [8]:
from sklearn.linear_model import LassoCV 

# Can play around with alphas for tuning - should make sure to print to see effects
alphas = np.logspace(-10, 100, 200)
# print(alphas)

model = LassoCV(alphas=alphas) 
model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f} alpha={:0.3f}".format(r2, me, model.alpha_))


r2=0.806 MSE=722659.757 alpha=0.891


c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [9]:
from sklearn.linear_model import ElasticNetCV

alphas = np.logspace(-10, 100, 200)

model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=alphas) 
model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f}".format(r2,me))

c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


r2=0.806 MSE=723678.358


c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [10]:
# LassoCV as part of a pipeline, with some preprocessing

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

alphas = np.logspace(-10, 100, 200)

# PolynomialFeatures takes an input [a, b] and returns [1, a, b, a^2, ab, b^2]
model = Pipeline([
    ('poly', PolynomialFeatures(2)),
    ('lasso', LassoCV(alphas=alphas)),
])

model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f}, MSE={:0.3f}, alpha={:0.3f}".format(r2, me, model.named_steps['lasso'].alpha_))

c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


r2=0.867, MSE=495449.207, alpha=0.249


c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [11]:
alphas = np.logspace(-10, 100, 200)

model = Pipeline([
    ('poly', PolynomialFeatures(2)), 
    ('ridge', RidgeCV(alphas=alphas)),
])

model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f} alpha={:0.3f}".format(r2, me, model.named_steps['ridge'].alpha_))

r2=0.863 MSE=510936.215 alpha=0.249


In [12]:
# Same as above, just with 3 poly features
alphas = np.logspace(-10, 100, 200)

model = Pipeline([
    ('poly', PolynomialFeatures(3)), 
    ('ridge', RidgeCV(alphas=alphas)),
])

model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f} alpha={:0.3f}".format(r2, me, model.named_steps['ridge'].alpha_))

r2=0.849 MSE=564341.229 alpha=84066.529


In [13]:
alphas = np.logspace(-10, 100, 200)

model = Pipeline([
    ('poly', PolynomialFeatures(4)), 
    ('ridge', RidgeCV(alphas=alphas)),
])

model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f} alpha={:0.3f}".format(r2, me, model.named_steps['ridge'].alpha_))

r2=0.815 MSE=687767.208 alpha=2221946860.940


In [14]:
alphas = np.logspace(-10, 100, 200)

model = Pipeline([
    ('poly', PolynomialFeatures(4)), 
    ('lasso', LassoCV(alphas=alphas)),
])

model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f} alpha={:0.3f}".format(r2, me, model.named_steps['lasso'].alpha_))

c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


r2=0.775 MSE=837415.128 alpha=23542.864


c:\econsw\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [15]:
# RF regression
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor() 
model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f}".format(r2,me))

r2=0.905 MSE=353977.107


# Saving models

In [16]:
# Saving a model
with open('bikeshare-rf.pkl', 'wb') as f:
    pickle.dump(model, f)

In [17]:
# Loading a saved model
with open('bikeshare-rf.pkl', 'rb') as f:
    model = pickle.load(f)

In [18]:
model.predict(X_test)

array([6983.2, 6493.4, 1707. , 2889.7, 7184.2, 1448.4, 4955.7, 5556.1,
       1925. , 4646.2, 5513. , 6476. , 4546.1, 3522.2, 4690. , 7605.7,
       4970.2, 1719.1, 6399.4, 2485.1, 1769.3, 1514.3, 3661.6, 3978.2,
       1755. , 5495.4, 5324.2, 3691.9, 6596.9, 5740.2, 5285.3, 2240. ,
       3824.2, 4519.9, 7099.3, 3499.3, 3783.3, 1359.9, 4884.1, 1486.6,
       2894.8, 3731. , 4911.8, 6547.3, 6465.6, 6115.5, 2914.1, 3071.5,
       2073.6, 4053.6, 3371.6, 6762.7, 6955.4, 1684.9, 5994.4, 2071.7,
       4848.7, 2230.4, 1953.7, 7664.7, 1809.5, 3649.4, 7449.7, 5208.9,
       6559.2, 4317.9, 3537.5, 6389.7, 4781.3, 4281.7, 2472.4, 5171.4,
       7594.5, 3368.8, 6214.1, 8243.6, 3441.6, 6484.6, 1296.8, 4663.9,
       2977.8, 7586.9, 4220.9, 6687.7, 5390.4, 3766.3, 6632.8, 5106.5,
       1892. , 2080. , 5322.8, 7183. , 5226.4, 2332.6, 5044.5, 4790.3,
       1639.6, 6030.2, 5392.9, 3013.8, 1608.7, 1432.5, 6714.8, 4242. ,
       5310.9, 4565.1, 1943.5, 1618.3, 2799.4, 5132.4, 6736.6, 3914.2,
      

In [19]:
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor() 
model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f}".format(r2, me))

r2=0.849 MSE=561338.399


In [20]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge() 
model.fit(X_train, y_train)

yhat = model.predict(X_test)

r2 = r2_score(y_test, yhat)
me = mse(y_test, yhat)

print("r2={:0.3f} MSE={:0.3f}".format(r2, me))

r2=0.806 MSE=722330.243


In [21]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(5)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7598311925153668